In [ ]:
!wget https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!tar -xvf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!ls spark-3.1.2-bin-hadoop2.7/

bin   data	jars	    LICENSE   NOTICE  R		 RELEASE  yarn
conf  examples	kubernetes  licenses  python  README.md  sbin


In [ ]:
!ls /usr/lib/jvm

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
!ls /content/spark-3.1.2-bin-hadoop2.7

In [ ]:
!pip install findspark

In [ ]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext
sparkConfig = SparkConf().setAppName("RDDs")
sc = SparkContext.getOrCreate(conf = sparkConfig)


# **Project**

In [ ]:
!wget "https://raw.githubusercontent.com/AISCIENCES/course-master-big-data-with-pyspark-and-aws/main/Code/02-Spark%20RDDs/StudentData.csv"

--2021-08-18 13:48:13--  https://raw.githubusercontent.com/AISCIENCES/course-master-big-data-with-pyspark-and-aws/main/Code/02-Spark%20RDDs/StudentData.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75701 (74K) [text/plain]
Saving to: ‘StudentData.csv’

StudentData.csv     100%[===================>]  73.93K  --.-KB/s    in 0.01s   

2021-08-18 13:48:14 (5.98 MB/s) - ‘StudentData.csv’ saved [75701/75701]



In [ ]:
data = sc.textFile("/content/StudentData.csv")
header = data.first()
data.take(5)

['age,gender,name,course,roll,marks,email',
 '28,Female,Hubert Oliveras,DB,02984,59,Annika Hoffman_Naoma Fritts@OOP.com',
 '29,Female,Toshiko Hillyard,Cloud,12899,62,Margene Moores_Marylee Capasso@DB.com',
 '28,Male,Celeste Lollis,PF,21267,45,Jeannetta Golden_Jenna Montague@DSA.com',
 '29,Female,Elenore Choy,DB,32877,29,Billi Clore_Mitzi Seldon@DB.com']

In [ ]:
dataRDD = data.filter(lambda x: x != header)

Show the number of students in the file

In [ ]:
dataRDD.count()

1000

Show the total marks achieved by Female and Male students

In [ ]:
dataRDD.map(lambda x: (x.split(',')[1], int(x.split(',')[-2]))).reduceByKey(lambda x,y: x+y).collect()

[('Female', 29636), ('Male', 30461)]

Show the total no. of students who have passed and failed. 50+ == pass

The ones' who passed

In [ ]:
dataRDD.map(lambda x: x.split(',')).filter(lambda x: int(x[-2]) > 50).count()

630

The ones' who failed

In [ ]:
dataRDD.map(lambda x: x.split(',')).filter(lambda x: int(x[-2]) <= 50).count()

370

Show the total number of student enrolled per course

In [ ]:
dataRDD.map(lambda x: x.split(',')[3]).countByValue()

defaultdict(int,
            {'Cloud': 192,
             'DB': 157,
             'DSA': 176,
             'MVC': 157,
             'OOP': 152,
             'PF': 166})

Total marks students have achieved per course

In [ ]:
dataRDD.map(lambda x: (x.split(',')[3], int(x.split(',')[-2]))).reduceByKey(lambda x,y: x+y).collect()

[('DB', 9270),
 ('Cloud', 11443),
 ('PF', 9933),
 ('MVC', 9585),
 ('OOP', 8916),
 ('DSA', 10950)]

Average marks students have achieved per course

In [ ]:
dataRDD.map(lambda x: (x.split(',')[3], (int(x.split(',')[-2]),1))).reduceByKey(lambda x,y: (x[0]+y[0] , x[1]+y[1]))\
.map(lambda x: (x[0], x[1][0]/x[1][1])).collect()

[('DB', 59.044585987261144),
 ('Cloud', 59.598958333333336),
 ('PF', 59.83734939759036),
 ('MVC', 61.05095541401274),
 ('OOP', 58.6578947368421),
 ('DSA', 62.21590909090909)]

Min marks achieved per course

In [ ]:
dataRDD.map(lambda x: (x.split(',')[3], int(x.split(',')[-2]))).reduceByKey(lambda x,y: x if x<y else y).collect()

[('DB', 20), ('Cloud', 20), ('PF', 20), ('MVC', 22), ('OOP', 20), ('DSA', 20)]

Max marks achieved per course

In [ ]:
dataRDD.map(lambda x: (x.split(',')[3], int(x.split(',')[-2]))).reduceByKey(lambda x,y: x if x>y else y).collect()

[('DB', 98), ('Cloud', 99), ('PF', 99), ('MVC', 99), ('OOP', 99), ('DSA', 99)]

Avg age of female and male students

In [ ]:
dataRDD.map(lambda x: (x.split(',')[1], (int(x.split(',')[0]),1))).reduceByKey(lambda x,y: (x[0]+y[0] , x[1]+y[1]))\
.map(lambda x: (x[0], x[1][0]/x[1][1])).collect()

[('Female', 28.489021956087825), ('Male', 28.52304609218437)]